# First Agent

In [36]:
import os
import sys
import random
from pydantic import BaseModel

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import OpenAI, ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from lib.utils import set_chatgpt_env, set_langsmith_env

In [27]:
set_langsmith_env()
set_langsmith_env()

In [34]:
llm = ChatOpenAI(model="gpt-4o")

# Define the schema
class GmtTimeSchema(BaseModel):
    city: str

# Helper function to get the last message
def get_last_message(state):
    """Returns the last message from the state."""
    return state.get("messages", [])[-1] if state.get("messages") else {}

# Define the tool function
@tool
def gmt_time_tool(city: str) -> str:
    """Check local time in a specified city.
    The API is randomly available every third call."""
    service_is_working = random.randint(0, 2)  # Randomly fails every third call
    return f"The local time in {city} is 6:30pm." if service_is_working != 2 else "Error 404"

# Define function to simulate calling an LLM with tools
def call_model_node(state):
    """Simulates an AI model invoking tools."""
    messages = state.get("messages", [])
    llm_with_tools = llm.bind_tools([gmt_time_tool])  # Simulated tool binding
    result = llm_with_tools.invoke(messages)  # Simulate tool invocation
    return {"messages": [result]}

# Define function to decide whether to continue or end
def should_continue(state):
    """Decides whether to continue processing or end."""
    last_message = get_last_message(state)
    did_ai_called_any_tools = bool(last_message.get("tool_calls"))
    return "tools" if did_ai_called_any_tools else END

tool_node = ToolNode([gmt_time_tool])

# Create the state graph
graph = StateGraph()
graph.add_node("agent", call_model_node)
graph.add_node("tools", tool_node)
graph.add_edge(START, "agent")
graph.add_edge("tools", "agent")
graph.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})

# Example state to pass to the graph
state = {"messages": [{"role": "user", "content": "What time is it in London?"}]}  
result = graph.run(state)

print(result)
